In [1]:
import pandas as pd

# Consulta CNPJ por socio
This is a function called Consulta_SOCIO_arquivo that takes two parameters as inputs: arq_csv and socio_input.

The function starts by defining a list nome_colunas that contains the names of the columns of the DataFrame. It then creates a variable col_filter that is assigned the value of the list nome_colunas. The variable chunk_size is set to 1,000,000 and an empty DataFrame df_output is created.

The function then uses a for loop to iterate through the chunks of the large CSV file arq_csv. For each chunk, the columns are assigned the names specified in the list nome_colunas. Then it filters the chunk using only the columns in the col_filter and drops the rows that have NaN values in the 'NOME_SOCIO_RAZAO_SOCIAL' column. After that, it filters the chunk again using the socio_input parameter to find the rows that contain the input string in the 'NOME_SOCIO_RAZAO_SOCIAL' column. Finally, the chunks are concatenated to the df_output DataFrame.

The function then prints a message with the searched name and return the final DataFrame, df_output, that contains all the rows that match the filter conditions from all chunks of the large CSV file.

This function is useful to filter large CSV files that cannot fit in the memory and the search is based on the column 'NOME_SOCIO_RAZAO_SOCIAL' and 'socio_input' parameter.





In [11]:
def Consulta_SOCIO_arquivo( arq_csv:str, socio_input:str)->pd.DataFrame:
    """
    The function receives two parameters:
    arq_csv (str) : the path of the csv file to be read
    socio_input (str) : a string to filter the name of the column 'NOME_SOCIO_RAZAO_SOCIAL'
    
    The function filter large CSV files that cannot fit in the memory and the search is based on the column 'NOME_SOCIO_RAZAO_SOCIAL' and 'socio_input' parameter.
    It starts by defining a list 'nome_colunas' that contains the names of the columns of the DataFrame.
    It then creates a variable 'chunk_size' that is set to 1,000,000 and an empty DataFrame 'df_output' is created.
    The function then uses a for loop to iterate through the chunks of the large CSV file 'arq_csv'.
    For each chunk, the columns are assigned the names specified in the list 'nome_colunas'.
    Then it filters the chunk using only the columns and drops the rows that have NaN values in the 'NOME_SOCIO_RAZAO_SOCIAL' column.
    After that, it filters the chunk again using the 'socio_input' parameter to find the rows that contain the input string in the 'NOME_SOCIO_RAZAO_SOCIAL' column.
    Finally, the chunks are concatenated to the 'df_output' DataFrame.
    The function then prints a message with the searched name and return the final DataFrame, 'df_output', that contains all the rows that match the filter conditions from all chunks of the large CSV file.
    """
    
    nome_colunas  =  ['CNPJ_BASICO', 'IDENTIFICADOR_SOCIO','NOME_SOCIO_RAZAO_SOCIAL','CPF_CNPJ_SOCIO','QUALIFICACAO_SOCIO',
             'DT_ENTRADA_SOCIEDADE','PAIS','REPRESENTANTE_LEGAL','NOME_REPRESENTANTE','QUALIFICACAO_REPRESENTANTE_LEGAL',
             'FAIXA_ETARIA']
    
    chunk_size = 1000000
    df_output = pd.DataFrame()
    socio_input = socio_input.upper()
    
    for chunk in pd.read_csv(arq_csv, sep=';' ,  header= None , encoding='ISO-8859-1', low_memory=False, chunksize = chunk_size, iterator = True, na_values = None):
        chunk.columns = nome_colunas
        chunk=chunk.dropna(subset=['NOME_SOCIO_RAZAO_SOCIAL'])
        chunk = chunk[chunk['NOME_SOCIO_RAZAO_SOCIAL'].str.contains(socio_input)]
        df_output = pd.concat([df_output,chunk ], ignore_index=True)
     
    
    return df_output
# =======================================================================================================================
def Consulta_SOCIO_todos_arquivos(socio_pesquisado, lista_arquivos_socios):  
    df_output = pd.DataFrame()
    for ARQUIVO in lista_arquivos_socios:
        df_output = pd.concat([df_output , Consulta_SOCIO_arquivo(ARQUIVO,socio_pesquisado)], axis=0)
    
    return df_output

# Consulta os CNPJs encontrados na pesquisa por socio

In [3]:
# le um arquivo generico,importa o header e filtra as colunas e consulta o cnpj
def Consulta_cnpj_arquivo(arq_csv , cnpj , nome_colunas,  col_filter ):
    chunk_size = 1000000
    df_output = pd.DataFrame()
    
    for chunk in pd.read_csv(arq_csv, sep=';' ,  header= None , encoding='ISO-8859-1', low_memory=False, chunksize = chunk_size, iterator = True):
        chunk.columns = nome_colunas
        chunk=chunk[col_filter]
        chunk = chunk.loc[chunk['CNPJ_BASICO'].isin(cnpj)]
        df_output = pd.concat([df_output,chunk ],axis = 0)
       
    return df_output


# ================================================================================================================

def Consulta_cnpj_lista_arquivos(input_cnpj , lista_arquivos, header_arquivos , filtro_colunas):
   
    df_output=pd.DataFrame()
      
    
    for ARQUIVO in lista_arquivos:
           df_output = pd.concat([df_output,
                                              Consulta_cnpj_arquivo(arq_csv=ARQUIVO,
                                                                                 cnpj=input_cnpj,
                                                                                 nome_colunas=header_arquivos,
                                                                                 col_filter=filtro_colunas)],
                                 axis=0)

    df_output.fillna('-')
    return df_output


# Formatando o cruzamneto das bases empresas, socios e estabs

In [4]:

def tabelas_complementares(estabelecimentos, socios, empresas):
    # Tabelas auxiliares
    cidades = pd.read_csv('a.Municipios/Municipios.csv', header= None , encoding='ISO-8859-1', sep=';')
    cidades.columns = ['MUNICIPIO','nome_municipio']

    naturezas  = pd.read_csv('a.Naturezas/naturezas.csv', header= None , encoding='ISO-8859-1', sep=';')
    naturezas.columns= ['NATUREZA_JURIDICA','desc_natureza_juridica']

    qualificacoes = pd.read_csv('a.Qualificacoes/qualificacoes.csv', header= None , encoding='ISO-8859-1', sep=';')
    qualificacoes.columns = ['QUALIFICACAO_SOCIO','desc_qualificacao']
      
    cnae = pd.read_csv('a.Cnaes/cnaes_descricao.csv', header= None , encoding='ISO-8859-1', sep=';')
    cnae.columns = ['CNAE_PRINCIPAL','descricao_cnae']
    
    # tabela com a descrição da faixa etaria dos socios
    index = [0,1,2,3,4,5,6,7,8,9]
    faixa_idade =[ 'Não se aplica', 'entre 0 a 12 anos', 'entre 13 a 20 anos','entre 21 a 30 anos','entre 31 a 40 anos',
    'entre 41 a 50 anos','entre 51 a 60 anos','entre 61 a 70 anos','entre 71 a 80 anos','maiores de 80 anos' ]
    df_faixa_etaria = pd.DataFrame({'FAIXA_ETARIA': index,
                                    'faixa_idade_socio':faixa_idade})

    # tabela com a descrição se o socio é PJ, PF ou estrangeiro
    ind =[1,2,3]
    desc_ind = ['Pessoa Física','Pessoa Jurídica','Estrangeiro']
    indenficador_socio = pd.DataFrame({'IDENTIFICADOR_SOCIO':ind,
                                       'id_socio_descricao':desc_ind})

    # tabela para identicar matriz e filial
    matriz_filial = pd.DataFrame({'ID_MATRIZ_FILIAL':[1,2],
                              'matriz_filial':['Matriz','Filial']})
    # porte da empresa
    porte_emp =  pd.DataFrame({'PORTE_EMPRESA':[0,1,3,5],
                              'Desc_Porte_empresa':['Não Informado','Micro','Pequeno porte','Não Micro/Pequena empresa']})
    # porte da empresa
    situacao_cadast =  pd.DataFrame({'SITUACAO_CADASTRAL':[1,2,3,4,8],
                              'Desc_Situacao_Cadastral':['Nula','Ativa','Suspensa','Inapta','Baixada']})
    
# --------------------------------------------------------------------------------------------------------------------------
    # fazendo os JOIN's das tabelas estab, empresas e socios
    df= pd.merge(socios,empresas, on = 'CNPJ_BASICO')
    df = pd.merge(df,estabelecimentos, on = 'CNPJ_BASICO')

    # fazendo os JOIN's com as tabelas complemetares
    df = pd.merge(df,cidades, on ='MUNICIPIO')
    df = pd.merge(df,indenficador_socio, on ='IDENTIFICADOR_SOCIO')
    df = pd.merge(df,df_faixa_etaria, on ='FAIXA_ETARIA')
    df = pd.merge(df,matriz_filial, on = 'ID_MATRIZ_FILIAL')
    df = pd.merge(df,cnae, on = 'CNAE_PRINCIPAL')
    df = pd.merge(df,naturezas, on = 'NATUREZA_JURIDICA')
    df = pd.merge(df,qualificacoes, on = 'QUALIFICACAO_SOCIO')
    df = pd.merge(df,porte_emp, on = 'PORTE_EMPRESA')
    df = pd.merge(df,situacao_cadast, on = 'SITUACAO_CADASTRAL')

# --------------------------------------------------------------------------------------------------------------------------
   
    # formatando as datas
    df['Data_ini_Atividade'] = df['DT_INICIO_ATIVIDADE'].astype(str).apply(lambda x : x[6:]) + '/' + \
                                df['DT_INICIO_ATIVIDADE'].astype(str).apply(lambda x : x[4:6]) + '/' + \
                                df['DT_INICIO_ATIVIDADE'].astype(str).apply(lambda x : x[0:4])
    
    df['Data_entrada_sociedade'] = df['DT_ENTRADA_SOCIEDADE'].astype(str).apply(lambda x : x[6:]) + '/' + \
                                    df['DT_ENTRADA_SOCIEDADE'].astype(str).apply(lambda x : x[4:6]) + '/' + \
                                    df['DT_ENTRADA_SOCIEDADE'].astype(str).apply(lambda x : x[0:4])
    
    df['Data_situacao_cadastral'] = df['DT_SITUACAO_CADASTRAL'].astype(str).apply(lambda x : x[6:]) + '/' + \
                                    df['DT_SITUACAO_CADASTRAL'].astype(str).apply(lambda x : x[4:6]) + '/' + \
                                    df['DT_SITUACAO_CADASTRAL'].astype(str).apply(lambda x : x[0:4])

# --------------------------------------------------------------------------------------------------------------------------
    # selecionando as variaveis 
    cols = ['CNPJ_BASICO','matriz_filial','RAZAO_SOCIAL_NOME_EMPRESARIAL', 'NOME_FANTASIA', 'Desc_Situacao_Cadastral', 
            'Desc_Porte_empresa','CAPITAL_SOCIAL', 'Data_ini_Atividade','NOME_SOCIO_RAZAO_SOCIAL', 'CPF_CNPJ_SOCIO',
            'desc_qualificacao','faixa_idade_socio','Data_entrada_sociedade','CNAE_PRINCIPAL', 'descricao_cnae',
            'TIPO_LOGRADOURO', 'LOGRADOURO', 'NUMERO', 'COMPLEMENTO', 'BAIRRO','CEP', 'UF', 'MUNICIPIO',
            'DDD1', 'TELEFONE1', 'DDD2', 'TELEFONE2','EMAIL',  'nome_municipio', 'desc_natureza_juridica']
    df = df[cols]
    return df

# Pesquisa um nome de socio e gera a tabela completa com as empresas desse socio

In [ ]:
def Pesquisa_por_socio(socio_pesquisado):
    import time
    
    start_time= time.time()
    #    --------------------------------------------------------------------------------------------------

    # INPUTS SOCIOS
    arquivos_socios = ['K3241.K03200Y0.D210080.SOCIOCSV.csv','K3241.K03200Y1.D210081.SOCIOCSV.csv',
                      'K3241.K03200Y2.D210082.SOCIOCSV.csv','K3241.K03200Y3.D210083.SOCIOCSV.csv',
                      'K3241.K03200Y4.D210084.SOCIOCSV.csv','K3241.K03200Y5.D210085.SOCIOCSV.csv',
                      'K3241.K03200Y6.D210086.SOCIOCSV.csv','K3241.K03200Y7.D210087.SOCIOCSV.csv',
                      'K3241.K03200Y8.D210088.SOCIOCSV.csv','K3241.K03200Y9.D210089.SOCIOCSV.csv']

    socios = Consulta_SOCIO_todos_arquivos(socio_pesquisado=socio_pesquisado,lista_arquivos_socios = arquivos_socios)
    print('Arquivo Socios: OK')
    #    --------------------------------------------------------------------------------------------------

    # INPUTS EMPRESAS

    CNPJ_ = socios['CNPJ_BASICO']

    arquivo_empresas = ['K3241.K03200Y0.D210080.EMPRECSV.csv','K3241.K03200Y1.D210081.EMPRECSV.csv',
            'K3241.K03200Y2.D210082.EMPRECSV.csv','K3241.K03200Y3.D210083.EMPRECSV.csv',
            'K3241.K03200Y4.D210084.EMPRECSV.csv','K3241.K03200Y5.D210085.EMPRECSV.csv',
            'K3241.K03200Y6.D210086.EMPRECSV.csv','K3241.K03200Y7.D210087.EMPRECSV.csv',
            'K3241.K03200Y8.D210088.EMPRECSV.csv','K3241.K03200Y9.D210089.EMPRECSV.csv']


    header  =  ['CNPJ_BASICO', 'RAZAO_SOCIAL_NOME_EMPRESARIAL','NATUREZA_JURIDICA','QUALIFICACAO_RESPONSAVEL',
            'CAPITAL_SOCIAL','PORTE_EMPRESA','ENTE_FEDERATIVO_RESPONSAVEL']

    # RODANDO A CONSULTA
    empresas = Consulta_cnpj_lista_arquivos(input_cnpj=CNPJ_ , 
                                lista_arquivos=arquivo_empresas , 
                                header_arquivos=header , 
                                filtro_colunas=header)
    print('Arquivo Empresas: OK')
    #    --------------------------------------------------------------------------------------------------

    # INPUTS ESTABELECIMENTOS
    CNPJ_ = socios['CNPJ_BASICO']

    arquivos_estabelecimento = ['K3241.K03200Y0.D21119.ESTABELE.csv',
                                'K3241.K03200Y1.D21119.ESTABELE.csv',# esse arquivo é que tava faltando
                                'K3241.K03200Y2.D21119.ESTABELE.csv',
                                'K3241.K03200Y3.D21119.ESTABELE.csv',
                                'K3241.K03200Y4.D21119.ESTABELE.csv',
                                'K3241.K03200Y5.D21119.ESTABELE.csv',# esse arquivo é que tava faltando
                                'K3241.K03200Y6.D21119.ESTABELE.csv',
                                'K3241.K03200Y7.D21119.ESTABELE.csv',
                                'K3241.K03200Y8.D21119.ESTABELE.csv',
                                'K3241.K03200Y9.D21119.ESTABELE.csv']
 

    header  =  [ 'CNPJ_BASICO', 'CNPJ_ORDEM','CNPJ_DV','ID_MATRIZ_FILIAL','NOME_FANTASIA','SITUACAO_CADASTRAL',
        'DT_SITUACAO_CADASTRAL','MOTIVO_SITUACAO_CADASTRAL','NM_CIDADE_EXT','PAIS','DT_INICIO_ATIVIDADE',
        'CNAE_PRINCIPAL','CNAE_SECUNDARIO','TIPO_LOGRADOURO','LOGRADOURO','NUMERO', 'COMPLEMENTO','BAIRRO',
        'CEP', 'UF','MUNICIPIO','DDD1', 'TELEFONE1','DDD2', 'TELEFONE2','DDD_FAX', 'TELEFONE_FAX','EMAIL',
        'SITUACAO_ESPECIAL','DT_SITUACAO_ESPECIAL']


    # RODANDO A CONSULTA
    estabs = Consulta_cnpj_lista_arquivos(input_cnpj=CNPJ_ , 
                                lista_arquivos=arquivos_estabelecimento , 
                                header_arquivos=header , 
                                filtro_colunas=header)
    print('Arquivo Estabelecimentos: OK')
    
    end_time = time.time()
    print(f'O Programa levou {round(end_time-start_time,2)} segundos ou {round((end_time-start_time)/60,2)} minutos.')
    
    return tabelas_complementares(estabelecimentos=estabs,empresas=empresas,socios=socios)

# Fazendo a pesquisa

In [15]:
x = 'baptista garcia'
df = Pesquisa_por_socio(x)

Arquivo Socios: OK
Arquivo Empresas: OK
Arquivo Estabelecimentos: OK
O Programa levou 367.45 segundos ou 6.12 minutos.


In [16]:
df

,CNPJ_BASICO,matriz_filial,RAZAO_SOCIAL_NOME_EMPRESARIAL,NOME_FANTASIA,Desc_Situacao_Cadastral,Desc_Porte_empresa,CAPITAL_SOCIAL,Data_ini_Atividade,NOME_SOCIO_RAZAO_SOCIAL,CPF_CNPJ_SOCIO,...,CEP,UF,MUNICIPIO,DDD1,TELEFONE1,DDD2,TELEFONE2,EMAIL,nome_municipio,desc_natureza_juridica
0,33236496,Matriz,J B GARCIA FILHO CONVERTEDORA GNV LTDA,BOX 854 CONVERTEDORA GNV,Ativa,Micro,"100000,00",03/04/2019,JOAO BAPTISTA GARCIA FILHO,***537907**,...,25931858.0,RJ,5849,21.0,27393008,NaN,NaN,BOX854@GMAIL.COM,MAGE,Sociedade Empresária Limitada
1,46744557,Matriz,PRIME CORRETORA DE SEGUROS E CONSULTORIA LTDA,PRIME CORRETORA DE SEGUROS E CONSULTORIA,Ativa,Micro,"50000,00",10/06/2022,HELOISA DA ROCHA BAPTISTA GARCIA,***779088**,...,12922590.0,SP,6251,11.0,74010538,NaN,NaN,PRIMECORRETORAECONSULTORIA@GMAIL.COM,BRAGANCA PAULISTA,Sociedade Empresária Limitada
2,35457279,Matriz,BAPTISTA E PEREIRA ASSESSORIA E CONSULTORIA EM...,NaN,Ativa,Micro,"10000,00",08/11/2019,MARIO HENRIQUE BAPTISTA GARCIA,***838368**,...,1139003.0,SP,7107,11.0,48975026.0,NaN,NaN,ACRIALESSEPEREIRA@GMAIL.COM,SAO PAULO,Sociedade Empresária Limitada
3,46540770,Matriz,NEOSP ENGENHARIA LTDA,NEOSP,Ativa,Micro,"100000,00",25/05/2022,JOAN BAPTISTA GARCIA COLOME,***568178**,...,6454000.0,SP,6213,21.0,22222222,NaN,NaN,CONTATO@METACONTABIL.CNT.BR,BARUERI,Sociedade Empresária Limitada
4,4658690,Matriz,AUTO MECANICA RUA GUARANI EIRELI,BOX 854,Ativa,Micro,"88000,00",27/08/2001,JOAO BAPTISTA GARCIA FILHO,***537907**,...,25931858.0,RJ,5849,21,27393008,NaN,NaN,BOX854@GMAIL.COM,MAGE,Empresa Individual de Responsabilidade Limitad...
5,47172723,Matriz,MAMORI PARTICIPACOES INTERMEDIACOES E CONSULTO...,MAMORI PARTICIPACOES,Ativa,Pequeno porte,"10000,00",15/07/2022,FABIO BAPTISTA GARCIA,***885138**,...,6542110.0,SP,7047,11.0,84065514,NaN,NaN,FABIO.BGARCIA1990@GMAIL.COM,SANTANA DE PARNAIBA,Sociedade Empresária Limitada
6,2922769,Filial,SAO PAULINO AGROPECUARIA LTDA.,FAZENDA SAO PAULINO,Ativa,Pequeno porte,"0,00",15/06/1999,DIANA DE MEDEIROS BAPTISTA GARCIA,***659049**,...,86430000.0,PR,7859,43.0,7344346,NaN,NaN,NaN,SANTO ANTONIO DA PLATINA,Sociedade Empresária Limitada
7,2922769,Filial,SAO PAULINO AGROPECUARIA LTDA.,FAZENDA SAO FRANCISCO,Ativa,Pequeno porte,"0,00",15/06/1999,DIANA DE MEDEIROS BAPTISTA GARCIA,***659049**,...,86400000.0,PR,7629,43.0,7344346,NaN,NaN,NaN,JACAREZINHO,Sociedade Empresária Limitada
8,11680976,Matriz,FAZENDA LITORAL RESTAURANTE LTDA,NaN,Ativa,Pequeno porte,"100000,00",09/03/2010,JOAO BAPTISTA GARCIA JUNIOR,***138498**,...,11726500.0,SP,6921,11.0,29571098,NaN,NaN,LITORALPLAZA@DIVINOFOGAO.COM.BR,PRAIA GRANDE,Sociedade Empresária Limitada
9,44248447,Matriz,ISOLATESCH QUIMICA INDUSTRIA E COMERCIO DE IMP...,ISOLATESCH QUIMICA LTDA,Ativa,Pequeno porte,"100000,00",16/11/2021,JOAN BAPTISTA GARCIA COLOME,***568178**,...,13284076.0,SP,7237,11.0,44070375,NaN,NaN,ASSESSORIACONTABILGOMES@GMAIL.COM,VINHEDO,Sociedade Empresária Limitada
